In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib notebook

In [23]:
import math

from kelly_criterion.kelly import log_return
from kelly_criterion.samplers import make_bernoulli_return_sampler
from kelly_criterion.samplers import make_uniform_return_sampler
from kelly_criterion.plots import plot_func
from kelly_criterion.plots_3d import plot_3d_func_on_simplex

# Multivariate Kelly Goes All-In

## Objective

The goal of this lab is to explore the feature observed in \[Finkelstein et al, 1981\] that while a Kelly strategy in a univariate setting will never invest the full fortune, it can do in a multivariate setting.

## Background

Example 3 of \[Finkelstein et al, 1981\] describes:

>The surprising fact is that for one variable X1 a gambler does not bet all his fortune no matter what b is, but he does bet all his fortune on two independent copies of X1 for b large enough. 

The example they give is $X, Y \sim U(-1, b)$ IID. As usual, we want to maximize $G(\mu, \nu) = \mathbb{E}\big[ \log(1+\mu X + \nu Y \big]$. They show that $G$ is maximized at $(\frac{1}{2},\frac{1}{2})$ for $b > \log(16) - 1 \approx 1.77$. That is, the Kelly strategy allocates the full capital.

## Bivariate Uniform

In the case considered in \[Finkelstein et al, 1981\], we plot $G$ first for $b < 1.77$:

In [3]:
samples = make_uniform_return_sampler(1.1, n=2)(1000)
plot_3d_func_on_simplex(lambda x, y: log_return([x, y], samples))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

the behaviour looks similar to the univariate case. When we increase $b$, though, we see

In [4]:
samples = make_uniform_return_sampler(2, n=2)(1000)
plot_3d_func_on_simplex(lambda x, y: log_return([x, y], samples))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

as described.

Varying $b$, we see the issue is about hitting the boundary of the domain within the domain of the log.

Despite investing the full capital, the probability of a total loss is still 0.

## Bivariate Bernoulli

The above example raises the question of what this looks like in the case of bivariate Bernoullis, where there is a finite probability of total loss.

Again, for small $p$ it looks as previously

In [5]:
samples = make_bernoulli_return_sampler(0.6, n=2)(1000)
plot_3d_func_on_simplex(lambda x, y: log_return([x, y], samples))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

as $p$ grows we see similar behaviour to the uniform case:

In [6]:
samples = make_bernoulli_return_sampler(0.9, n=2)(1000)
plot_3d_func_on_simplex(lambda x, y: log_return([x, y], samples))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

But to see what happens near the boundary, let $G(\lambda_1, \lambda_2) = \mathbb{E}\big[1+\lambda_1 X_1 + \lambda_2 X_2 \big]$ for $X_i \sim Bernoulli(p)$ IID.

Then by the same argument from \[Finkelstein et al, 1981\], if there's an optimum then convexity of $G$ means it must occur along the ridge $\lambda_1 = \lambda_2$. So consider:

$$
\begin{align}
G(\lambda) &= \mathbb{E}\big[ \log(1+\lambda (X_1 + X_2)) \big] \\
    &= p^2 \log(1+2\lambda) + q^2\log(1-2\lambda) \\
\\
G'(\lambda) &= \frac{p^2}{1+2\lambda} - \frac{q^2}{1-2\lambda} \\
\\
G''(\lambda) &= 0 \\
\end{align}
$$

which gives an optimum at $\lambda^*(p) = \frac{1}{2}\Big( \frac{2p^2}{2p^2-2p+1} - 1 \Big)$. So we have $\lambda^*(p) \uparrow \frac{1}{2} \text{ as } p \uparrow 1$, but $\lambda^*(p) < \frac{1}{2}$

In [24]:
plot_func(lambda p: 0.5*(2*p*p/(2*p*p-2*p+1) - 1), x_lower=0.5, x_upper=1)

alt.Chart(...)

So the null probability of ruin is indeed preserved, as expected.

And it's become clear I had mis-interpretted the quote starting this off. Obviously, the claim that a univariate Kelly strategy never goes all in is false. An obvious counter-example being $X_1 > 0$.

The significance must lie more in the way convergence happens at the boundary in this continuous case and how that drives the behaviour there.

## References

[\[Finkelstein et al, 1981\]](https://www.jstor.org/stable/1426692?seq=1) Finkelstein, Mark, and Robert Whitley. "Optimal Strategies for Repeated Games." Advances in Applied Probability 13, no. 2 (1981): 415-28. Accessed December 9, 2020. doi:10.2307/1426692. https://escholarship.org/uc/item/4w69h2xx
